In [ ]:
from openai import OpenAI
import pandas as pd
import base64
import matplotlib.pyplot as plt
import seaborn as sns
import os

In [ ]:
api_key = input("Please enter the OpenAI API Key:")
# api_key = ""

client = OpenAI(api_key = api_key)

job_id = "ENTER JOB ID HERE"
job = client.fine_tuning.jobs.retrieve(job_id)

cwd = os.getcwd()

In [ ]:
# Run fine-tuned model

prompt = '''
Given the following APIs: 
"""
velocity_publisher(linear, angular): returns nothing. Publishes one linear and angular velocity command to the robot. 
stop(): returns nothing. Stops the robot. 
math.pi: returns the value of pi (float).
time.time(): returns the elapsed seconds since the epoch.
pedestrian_observed(): returns a boolean value indicating whether a pedestrian is observed.
"""
Write a Python function called "turn_right_60_degrees" that controls a car through a complete 60-degree right turn. Please response the python function only.
'''

ft_model = job.fine_tuned_model
completion = client.chat.completions.create(
  model=ft_model,
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
  ]
)
print(completion.choices[0].message.content)

### Out of domain task

In [ ]:
prompt = '''
Given the following APIs: 
"""
navigate(): returns nothing. Publish command to move the robot. 
stop(): returns nothing. Stop the robot. 
person_observed(): returns a boolean value indicating whether a person is observed.
target_observed(): returns a boolean value indicating whether the final target is observed.
signal(): returns nothing. Send a signal to users once observing the final target.
"""
Write a Python function called "search" that controls a robot to search for the final target. Please response the python function only.
'''

job_id = "ENTER JOB ID HERE"
job = client.fine_tuning.jobs.retrieve(job_id)

ft_model = job.fine_tuned_model
completion = client.chat.completions.create(
  model=ft_model,
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
  ]
)
print(completion.choices[0].message.content)

In [ ]:
with open('sample_inputs/spot_controller.py', mode="w") as f:
    f.write(completion.choices[0].message.content)
    f.close()

In [ ]:
# Run baseline model
cwd = os.getcwd()

prompt = '''
Given the following APIs: 
"""
navigate(): returns nothing. Publish command to move the robot. 
stop(): returns nothing. Stop the robot. 
person_observed(): returns a boolean value indicating whether a person is observed.
target_observed(): returns a boolean value indicating whether the final target is observed.
signal(): returns nothing. Send a signal to users once observing the final target.
"""
Write a Python function called "search" that controls a robot to search for the final target. Please response the python function only.
'''
completion = client.chat.completions.create(
  model='gpt-4o-mini',
  messages=[
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": prompt}
  ]
)
print(completion.choices[0].message.content)

In [ ]:
code = completion.choices[0].message.content
code = code[code.find('def'):]
code = code[:code.find("```")]
with open('sample_inputs/spot_benchmark_controller.py', mode="w") as f:
    f.write(code)
    f.close()